In [ ]:
# Install datasets for accessing the datasets
!pip install datasets

ignore the Error: ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

# Train a Logistic Regression Classifier for Multi-choice Question Answering



In this section, we implemented a Logistic Regression text classifier for the question answering task from scratch.

We will use the [CommonsenseQA](https://www.tau-nlp.sites.tau.ac.il/commonsenseqa) dataset for this question. Each of the data entry has four fields

- `id`
- `question`
- `choices`
    - `text`: all answer choices
    - `label`: the corresponding labels
- `answerKey`: the correct answer

In [ ]:
# Download the data
from datasets import load_dataset
csqa_dataset = load_dataset("tau/commonsense_qa")

print(csqa_dataset["train"][0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'id': '075e483d21c29a511267ef62bedc0461', 'question': 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?', 'question_concept': 'punishing', 'choices': {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['ignore', 'enforce', 'authoritarian', 'yell at', 'avoid']}, 'answerKey': 'A'}


In [ ]:
csqa_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 9741
    })
    validation: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 1221
    })
    test: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 1140
    })
})

In [ ]:
csqa_dataset['train']

Dataset({
    features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
    num_rows: 9741
})

In [ ]:
csqa_dataset['train'][0]

{'id': '075e483d21c29a511267ef62bedc0461',
 'question': 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?',
 'question_concept': 'punishing',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
  'text': ['ignore', 'enforce', 'authoritarian', 'yell at', 'avoid']},
 'answerKey': 'A'}

In [ ]:
csqa_dataset['validation']

Dataset({
    features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
    num_rows: 1221
})

## Data Preprocessing

In this section, we outline the construction of question-answering examples required for model training. Each entry in the CommonsenseQA dataset comprises a question and its corresponding answer choices. When working with large language models, a single input string containing both the question and all answer choices could suffice. However, simpler text classifiers are unable to process such complex input-output relationships. Therefore, we must preprocess the dataset to reformat it into a structure suitable for classification tasks.

One effective approach is to frame the question-answering task as a binary classification problem for each answer choice. This requires associating each data point with a binary label (`0` or `1`). To achieve this, we transform the dataset by generating an example for every `(question, answer_choice)` pair. Each example consists of the following:

1. **An input string**: The question concatenated with an individual answer choice.
2. **A binary label**: `1` if the answer choice is correct, and `0` otherwise.

### Example Dataset Transformation

Consider the example:  
**Original data**:  
`{"question": "Sammy wanted to go to where the people were. Where might he go?", "answer_choices": ["race track", "populated areas", "the desert", "apartment", "roadblock"]}`

We generate five separate data points from this example, where each input is a combination of the question and one answer choice:

1. `"Sammy wanted to go to where the people were. Where might he go? race track"`
2. `"Sammy wanted to go to where the people were. Where might he go? populated areas"`
3. `"Sammy wanted to go to where the people were. Where might he go? the desert"`
4. `"Sammy wanted to go to where the people were. Where might he go? apartment"`
5. `"Sammy wanted to go to where the people were. Where might he go? roadblock"`

The correct answer, `"populated areas"`, is assigned a label of `1`, while all other choices are labeled `0`. For clarity, we use dictionary notation to illustrate the final format (though in practice, input strings and labels will be stored in separate lists):

- `{input: "Sammy wanted to go to where the people were. Where might he go? race track", label: 0}`
- `{input: "Sammy wanted to go to where the people were. Where might he go? populated areas", label: 1}`
- `{input: "Sammy wanted to go to where the people were. Where might he go? the desert", label: 0}`
- `{input: "Sammy wanted to go to where the people were. Where might he go? apartment", label: 0}`
- `{input: "Sammy wanted to go to where the people were. Where might he go? roadblock", label: 0}`

This preprocessing step effectively reformats the dataset, enabling simple classifiers to handle the question-answering task as a binary classification problem.


### The Pre-process function to convert both training and validation set to the required format

In [ ]:
# to get the input and label lists
def preprocess(dataset):
    inputs = []
    labels = []

    # Iterate through the dataset
    for example in dataset:
        question = example['question']
        choices = example['choices']['text']
        labels_map = example['choices']['label']  # e.g. ["A", "B", "C", "D"]
        correct_label = example['answerKey']      # e.g. "B"

        # Find the index of the correct answer based on the label
        correct_index = labels_map.index(correct_label)

        # For each choice, create an input-output pair
        for i, choice in enumerate(choices):
            input_str = f"{question} {choice}"
            label = 1 if i == correct_index else 0

            inputs.append(input_str)
            labels.append(label)

    return inputs, labels

In [ ]:
# Process the train and validation sets
train_inputs, train_labels = preprocess(csqa_dataset['train'])
validation_inputs, validation_labels = preprocess(csqa_dataset['validation'])

In [ ]:
# Output the first question's input/output pairs
num_of_choices = len(csqa_dataset["train"][0]["choices"]["text"])
first_question_inputs = train_inputs[:num_of_choices]
first_question_labels = train_labels[:num_of_choices]
print("First question's input/output pairs:")
for input_str, label in zip(first_question_inputs, first_question_labels):
    print(f"Input: {input_str} | Label: {label}")

# Output the first question's dataset sizes
print(f"Training set size: {len(train_inputs)}")
print(f"Validation set size: {len(validation_inputs)}")

First question's input/output pairs:
Input: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change? ignore | Label: 1
Input: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change? enforce | Label: 0
Input: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change? authoritarian | Label: 0
Input: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change? yell at | Label: 0
Input: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change? avoid | Label: 0
Training set size: 48705
Validation set size: 6105


## Tokenization
Here we will tokenize the data using the okenizer provided by spaCy and generate the covab using CountVectorizer. Moreover, all words will be converted to lower case and we will filter out the tokens with their frequencies less than 3. After that both the `train` and `validation` sets will be convert to data metrices using the constructed vocab.

### Define the tokenizer using the one provided in spaCy lib

In [ ]:
from spacy.lang.en import English
nlp = English()

def tokenizer(s):
    tokenize = nlp.tokenizer
    return [token.text for token in tokenize(s)]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# use our own tokenizer, set min_df to 3 to filter out the tokens with their frequencies less than 3
vectorizer = CountVectorizer(tokenizer=tokenizer, lowercase=True, min_df=3, max_df=1.0, ngram_range=(1,1))
trn_data = vectorizer.fit_transform(train_inputs)
val_data = vectorizer.transform(validation_inputs)

# get the vocab generated by train_inputs
vocab = vectorizer.vocabulary_
# report size and dimension of vocab, two data metrices
print("Vocab size = {}".format(len(vocab)))
print("Size of training data = {}".format(trn_data.shape))
print("Size of validation data = {}".format(val_data.shape))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Vocab size = 9359
Size of training data = (48705, 9359)
Size of validation data = (6105, 9359)


## Build a Logistic Regression Classifier

build a Logistc Regression classifier and train a classification model with the datasets created.

In [ ]:
from sklearn.linear_model import LogisticRegression

# create a Logistic Regression Classifier model with l2 Regularization (λ=10)
lr_classifier = LogisticRegression(
    solver='liblinear',        # Use the liblinear solver
    fit_intercept=True,        # Include the bias term
    C=0.1,                     # C = 1 / λ, so C = 0.1 for λ=10
    penalty='l2',              # Use L2 regularization (default, but specified for clarity)
    max_iter=200
)
lr_classifier.fit(trn_data, train_labels)

LogisticRegression(C=0.1, max_iter=200, solver='liblinear')

In [ ]:
# The classification accuracies on both training and validation sets
print("Training accuracy = {}".format(lr_classifier.score(trn_data, train_labels)))
print("Validation accuracy = {}".format(lr_classifier.score(val_data, validation_labels)))

Training accuracy = 0.8
Validation accuracy = 0.8


This result accuracy is misleading, we will explain in the next section

## Accuracy of Question Answering

If we use the classification accuracy as the measure of the model's question answering performance, it can be misleading. For example, consider the question in section 2.1

|Input | Output | Predicted Label|
|:---|:---|:---|
|"Sammy wanted to go to where the people were. Where might he go? race track" | False | True |
|"Sammy wanted to go to where the people were. Where might he go? populated areas"| True | False |
|"Sammy wanted to go to where the people were. Where might he go? the desert" | False | False |
|"Sammy wanted to go to where the people were. Where might he go? apartment" | Fasle | False |
|"Sammy wanted to go to where the people were. Where might he go? roadblock" | Fasle | False |

If we measure the performance with the binary classification accuracy, the accuracy for this example group is 60%. However, if we measure the accuracy regarding the question answering performance, it is 0, because the model gives the wrong answer about the question. Therefore, to measure the performance of question answering, the evaluation metric has to aggregrate all predictions regarding the same question.

One further question is what if in the above example, a model gives two true prediction. To avoid cases like this, we need to pick the final prediction based on the prediction probability on $$p(y=True|x).$$ For example,

|Input | Output | Predicted Probability $p(y=True|x)$|
|:---|:---|:---|
|"Sammy wanted to go to where the people were. Where might he go? race track" | False | 50% |
|"Sammy wanted to go to where the people were. Where might he go? populated areas"| True | 90% |
|"Sammy wanted to go to where the people were. Where might he go? the desert" | False | 10% |
|"Sammy wanted to go to where the people were. Where might he go? apartment" | Fasle | 20% |
|"Sammy wanted to go to where the people were. Where might he go? roadblock" | Fasle | 10% |

With the given prediction probabilities, we should still consider the model prediction is `populated areas` (90%) instead of `race track` (50%).

In [ ]:
# the evaluation function

import numpy as np

def evaluation_function(classifier, data, original_dataset, true_labels):
    """
      :data: trn_data, val_data
      :original_dataset: arceasy_dataset['train'], arceasy_dataset['validation']
      :true_labels: train_labels, validation_labels
    """
    """
      !!!!! important !!!!!

      For each question in the dataset, we go through all the concatenated input of it,
      e.g. "Which of these is a primary color? red",
           "Which of these is a primary color? purple",
           ...

      For each concatenated input, we will check the predicted_probabilities table for
      the probability of this one being true.
      e.g. 60%, 90%, ...

      We will keep track of the global index (idx) of the highest one for each question.
      After going through all concatenated inputs of a question,
      we check if the one with the highest probability has ground truth label of True,
      this is a correct prediction, otherwise no.
    """

    # Predict probabilities for the data
    # [0] for label = 0, [1] for label = 1
    # e.g.
    # [[0.77829562 0.22170438]
    # [0.74739939 0.25260061]
    # [0.77748308 0.22251692]
    # ...
    # [0.70955397 0.29044603]
    # [0.74955357 0.25044643]
    # [0.70745351 0.29254649]]
    predicted_probabilities = classifier.predict_proba(data)


    idx = 0
    correct_predictions = 0

    # for each question
    for i in range(len(original_dataset)):
      # questions in this datasets may have different numbers of answer choices.
      choices_len = len(original_dataset[i]['choices']['text'])

      cur_max = 0
      max_idx = idx

      # go throught the concated input of this question. e.g. "Which of these is a primary color? red", ...
      for j in range(choices_len):

        if predicted_probabilities[idx][1] > cur_max:
          cur_max = predicted_probabilities[idx][1]
          max_idx = idx

        idx += 1

      # check if the max_idx for this question has ground truth label of True,
      # this is a correct prediction, otherwise no
      if true_labels[max_idx] == 1:
        correct_predictions += 1

    # Calculate accuracy
    # accuracy = correct_predictions / len(true_labels)
    accuracy = correct_predictions / len(original_dataset)

    return accuracy


In [ ]:
# Evaluate the classifier and report the results
training_score = evaluation_function(lr_classifier, trn_data, csqa_dataset['train'], train_labels)
validation_score = evaluation_function(lr_classifier, val_data, csqa_dataset['validation'], validation_labels)

print("Training accuracy = {}".format(training_score))
print("Validation accuracy = {}".format(validation_score))

Training accuracy = 0.312288266091777
Validation accuracy = 0.17117117117117117


Adjust the hyper-parameters of the classifier, and make sure it can produce a good performance.

In [ ]:
from sklearn.model_selection import ParameterGrid

def grid_search_2(classifier, params, trn_data, trn_labels, val_data, val_labels):
    best_params = None
    best_score = 0

    # Grid search
    for p in ParameterGrid(params):
        print("Evaluating params = {}".format(p))
        classifier.set_params(**p)
        classifier.fit(trn_data, trn_labels)

        # calculate accuracies on training and validation data
        train_score = evaluation_function(classifier, trn_data, csqa_dataset['train'], trn_labels)
        val_score = evaluation_function(classifier, val_data, csqa_dataset['validation'], val_labels)
        print(f"Training Accuracy: {train_score:.4f}, Validation Accuracy: {val_score:.4f}")

        # keep track of the best score and parameters
        if val_score > best_score:
            best_params = p
            best_score = val_score

    # Re-train the model with the best found parameters
    classifier.set_params(**best_params)
    classifier.fit(trn_data, trn_labels)

    print("Best params = {}".format(best_params))
    print("Best score = {}".format(best_score))

    return classifier, best_params, best_score

In [ ]:
params = {
    'penalty': ['l2'],
    'C': (0.01, 0.1, 1, 10, 100),
    'solver': ('lbfgs', 'sag', 'saga'),
    'fit_intercept': (True, False),
    'max_iter': (100, 200, 300),
    'multi_class': ('ovr', 'multinomial'),
    'tol': (1e-4, 1e-3, 1e-2)
}

In [ ]:
clf = LogisticRegression(max_iter=500)
clf = grid_search_2(clf, params, trn_data, train_labels, val_data, validation_labels)

Evaluating params = {'C': 0.01, 'fit_intercept': True, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'lbfgs', 'tol': 0.0001}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Training Accuracy: 0.2931, Validation Accuracy: 0.1982
Evaluating params = {'C': 0.01, 'fit_intercept': True, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'lbfgs', 'tol': 0.001}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Training Accuracy: 0.2866, Validation Accuracy: 0.2097
Evaluating params = {'C': 0.01, 'fit_intercept': True, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'lbfgs', 'tol': 0.01}


...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Training Accuracy: 0.3235, Validation Accuracy: 0.1507
Evaluating params = {'C': 100, 'fit_intercept': False, 'max_iter': 300, 'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'saga', 'tol': 0.01}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


Training Accuracy: 0.3210, Validation Accuracy: 0.1605
Best params = {'C': 0.01, 'fit_intercept': True, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'lbfgs', 'tol': 0.001}
Best score = 0.20966420966420968


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


### The training result of Logistic Regression Classifier
The grid search result above shows the best Logistic Regression Classifier has the validation accuracy of 0.20966420966420968 on the CommonsenseQA dataset, which was trained with the parameters of

-
{'C': 0.01, 'fit_intercept': True, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'lbfgs', 'tol': 0.001}

## Build a Neural Network Classifier
We will use the MLPClassifier provided by sklearn.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# create and configure the MLPClassifier with the specified hyperparameters
mlp_model = MLPClassifier(activation='tanh', solver='sgd', max_iter=500)

# train
mlp_model.fit(trn_data, train_labels)

MLPClassifier(activation='tanh', max_iter=500, solver='sgd')

In [ ]:
# The classification accuracies on both training and validation sets
print("Training accuracy = {}".format(mlp_model.score(trn_data, train_labels)))
print("Validation accuracy (dev) = {}".format(mlp_model.score(val_data, validation_labels)))

Training accuracy = 0.8
Validation accuracy (dev) = 0.8


### Use the graid search to train a Neural Network Classifier to get the best performance

In [ ]:
# Define the parameter grid for the MLPClassifier
params = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],        # Basic hidden layer configurations
    'activation': ['relu', 'tanh'],                        # Two commonly used activation functions
    'solver': ['adam', 'sgd'],                             # Two widely used solvers
    'alpha': [0.0001, 0.001],                              # Regularization term (L2 penalty)
    'max_iter': [200, 500]                                 # Number of iterations
}

In [ ]:
mlp = MLPClassifier(max_iter=500)
mlp = grid_search_2(mlp, params, trn_data, train_labels, val_data, validation_labels)

Evaluating params = {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'max_iter': 200, 'solver': 'adam'}
Training Accuracy: 0.9892, Validation Accuracy: 0.1876
Evaluating params = {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'max_iter': 200, 'solver': 'sgd'}
Training Accuracy: 0.2308, Validation Accuracy: 0.2039
Evaluating params = {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'max_iter': 500, 'solver': 'adam'}
Training Accuracy: 0.9892, Validation Accuracy: 0.1925
Evaluating params = {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'max_iter': 500, 'solver': 'sgd'}
Training Accuracy: 0.2417, Validation Accuracy: 0.2162
Evaluating params = {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'max_iter': 200, 'solver': 'adam'}
Training Accuracy: 0.9892, Validation Accuracy: 0.1802
Evaluating params = {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'max_iter': 200,

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training Accuracy: 0.9892, Validation Accuracy: 0.1736
Evaluating params = {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'max_iter': 200, 'solver': 'sgd'}
Training Accuracy: 0.2264, Validation Accuracy: 0.2056
Evaluating params = {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'max_iter': 500, 'solver': 'adam'}


### The training result of Neural Network Classifier
The grid search result above shows the best Neural Network Classifier has the validation accuracy of 0.2375 on the CommonsenseQA dataset, which was trained with the parameters of

-
{'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (100,), 'max_iter': 500, 'solver': 'sgd'}